In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds


In [ ]:
# Version check
import google.protobuf

print("TF:", tf.__version__)
print("TFDS:", tfds.__version__)
print("protobuf:", google.protobuf.__version__)


In [ ]:
# Dataset load
dataset, info = tfds.load("tf_flowers", with_info=True, as_supervised=True)
train_ds = dataset["train"]
print(info)


In [ ]:
# Preprocessing
IMG_SIZE = 224   # standard input size for many pretrained models
BATCH_SIZE = 32  # how many images to feed the model at once

# Function to resize and normalize each image
def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # resize to 224x224
    image = image / 255.0  # normalize pixel values [0,255] -> [0,1]
    return image, label

# Apply preprocessing to the dataset
train_ds = train_ds.map(format_example)

# Shuffle and batch the data
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Visualization of sample images
# Take one batch from the dataset
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    
    for i in range(9):  # show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())  # convert Tensor to NumPy for plotting
        plt.title(int(labels[i].numpy()))  # show class index as title
        plt.axis("off")


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, optimizers

# Number of flower classes (from dataset info)
num_classes = info.features["label"].num_classes
print("Number of classes:", num_classes)

# Load EfficientNetB0 base model (pretrained on ImageNet)
base_model = EfficientNetB0(
    weights="imagenet",        # use pretrained weights
    include_top=False,         # exclude the original ImageNet classifier
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze the base model (so its weights don't change initially)
base_model.trainable = False

# Build our custom model on top
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),        # reduce features
    layers.Dropout(0.5),                    # prevent overfitting
    layers.Dense(num_classes, activation="softmax")  # output layer
])

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Summary of the architecture
model.summary()


In [13]:
# Split dataset: 80% train, 20% validation
train_size = int(0.8 * len(list(train_ds.unbatch())))
val_size = int(0.2 * len(list(train_ds.unbatch())))

train_ds_split = train_ds.take(train_size)
val_ds_split = train_ds.skip(train_size).take(val_size)

print("Training samples:", train_size)
print("Validation samples:", val_size)


Training samples: 2936
Validation samples: 734


In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor="val_accuracy",    # stop if val accuracy doesn't improve
        patience=3,                # wait 3 epochs before stopping
        restore_best_weights=True  # roll back to best weights
    ),
    ModelCheckpoint(
        filepath="best_model.h5",  # save the best model to file
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
]


In [15]:
history = model.fit(
    train_ds_split,
    validation_data=val_ds_split,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
115/115 [==============================] - ETA: 0s - loss: 1.6444 - accuracy: 0.2041WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 90s 716ms/step - loss: 1.6444 - accuracy: 0.2041
Epoch 2/10
115/115 [==============================] - ETA: 0s - loss: 1.6296 - accuracy: 0.2253WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 83s 722ms/step - loss: 1.6296 - accuracy: 0.2253
Epoch 3/10
115/115 [==============================] - ETA: 0s - loss: 1.6231 - accuracy: 0.2401WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 83s 723ms/step - loss: 1.6231 - accuracy: 0.2401
Epoch 4/10
115/115 [==============================] - ETA: 0s - loss: 1.6362 - accuracy: 0.2136WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 82s 711ms/step - loss: 1.6362 - accuracy: 0.2136
Epoch 5/10
115/115 [==============================] - ETA: 0s - loss: 1.6311 - accuracy: 0.2223WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


...

115/115 [==============================] - 90s 778ms/step - loss: 1.6327 - accuracy: 0.2139
Epoch 7/10
115/115 [==============================] - ETA: 0s - loss: 1.6355 - accuracy: 0.2153WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 86s 747ms/step - loss: 1.6355 - accuracy: 0.2153
Epoch 8/10
115/115 [==============================] - ETA: 0s - loss: 1.6277 - accuracy: 0.2193WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 82s 713ms/step - loss: 1.6277 - accuracy: 0.2193
Epoch 9/10
115/115 [==============================] - ETA: 0s - loss: 1.6312 - accuracy: 0.2262WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 83s 721ms/step - loss: 1.6312 - accuracy: 0.2262
Epoch 10/10
115/115 [==============================] - ETA: 0s - loss: 1.6253 - accuracy: 0.2341WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


115/115 [==============================] - 82s 715ms/step - loss: 1.6253 - accuracy: 0.2341
